In [1]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import nbimporter
from function import combine_data
import pandas as pd
import time

In [2]:
re_scrape =[
    'https://www.hgtv.com/shows/house-hunters-international/episodes/1a',
    'https://www.hgtv.com/shows/house-hunters-international/episodes/100',
    'https://www.hgtv.com/shows/house-hunters-international/episodes/900',
    'https://www.hgtv.com/shows/house-hunters-international/episodes/3200',
    'https://www.hgtv.com/shows/house-hunters-international/episodes/5000',
    'https://www.hgtv.com/shows/house-hunters-international/episodes/6300',
    'https://www.hgtv.com/shows/house-hunters-international/episodes/9600',
    'https://www.hgtv.com/shows/house-hunters-international/episodes/10000',
    'https://www.hgtv.com/shows/house-hunters-international/episodes/10800',
    'https://www.hgtv.com/shows/house-hunters-international/episodes/11700',
    'https://www.hgtv.com/shows/house-hunters-international/episodes/12200',
    'https://www.hgtv.com/shows/house-hunters-international/episodes/12400',
    'https://www.hgtv.com/shows/house-hunters-international/episodes/12800',
    'https://www.hgtv.com/shows/house-hunters-international/episodes/12900',
    'https://www.hgtv.com/shows/house-hunters-international/episodes/145',
    'https://www.hgtv.com/shows/house-hunters-international/episodes/150',
    'https://www.hgtv.com/shows/house-hunters-international/episodes/152'
           ]

In [3]:
number_list = [] 
url_list = []
title_list = []
description_list = []

In [4]:
for url in re_scrape:
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)
    
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')

    episodes = soup.find('div', id = 'mod-episode-list-1')

    episode_number = episodes.find_all('span', class_='m-EpisodeCard__a-AssetInfo')
    episode_url = episodes.find_all('a')
    episode_description = episodes.find_all('p', class_='m-EpisodeCard__a-Description')

    numbers, urls, titles, descriptions = combine_data(episode_number, episode_url, episode_description)

    number_list = number_list + numbers
    url_list = url_list + urls
    title_list = title_list + titles
    description_list = description_list + descriptions
    
    browser.quit()
    
    time.sleep(1)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Dan\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Dan\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Dan\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Dan\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Dan\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


...

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Dan\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Dan\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Dan\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Dan\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\Dan\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [5]:
episode_dict = {}
episode_dict['Number'] = number_list
episode_dict['Title'] = title_list
episode_dict['Description'] = description_list
episode_dict['Link'] = url_list

episodes_df = pd.DataFrame(episode_dict)

In [6]:
episodes_df.drop_duplicates(subset = ['Title'])

episodes_df[['Season','Episode']] = episodes_df['Number'].str.split(', ', expand = True)

episodes_df['Season'] = episodes_df['Season'].map(lambda x:x.replace('Season ', ''))
episodes_df['Episode'] = episodes_df['Episode'].map(lambda x:x.replace('Episode ', '')).astype(int)
episodes_df

,Number,Title,Description,Link,Season,Episode
0,"Season 1, Episode 1",Costa Rica Vacation Home,Mark and Carolle Cotter love living in Northri...,www.hgtv.com/shows/house-hunters-international...,1,1
1,"Season 1, Episode 2",Dominican Vacation Home,"Throughout their fifteen years of marriage, St...",www.hgtv.com/shows/house-hunters-international...,1,2
2,"Season 1, Episode 3",Vacation Home in Spain,Lesley and Clive have been together for more t...,www.hgtv.com/shows/house-hunters-international...,1,3
3,"Season 1, Episode 4",Vacation Home in Italy,"Peter and Beverly Moule live in Goff's Oak, ju...",www.hgtv.com/shows/house-hunters-international...,1,4
4,"Season 1, Episode 5",Vacation Home in Paris,New Yorkers Jeff Ballinger and Mary Schiller r...,www.hgtv.com/shows/house-hunters-international...,1,5
...,...,...,...,...,...,...
205,"Season 152, Episode 9",Untapped Potential in Siem Reap,A globetrotting Australian fell in love with C...,www.hgtv.com/shows/house-hunters-international...,152,9
206,"Season 152, Episode 10",Cooking Up a Fresh Start in Cambodia,A couple and their dog are relocating within C...,www.hgtv.com/shows/house-hunters-international...,152,10
207,"Season 152, Episode 11",The Da Nang Dilemma,Newlyweds leave their comfortable lives in Aus...,www.hgtv.com/shows/house-hunters-international...,152,11
208,"Season 152, Episode 12",Bargaining in Bratislava,A young professional longing for a European ex...,www.hgtv.com/shows/house-hunters-international...,152,12


In [7]:
episodes_df.to_csv('House_Hunters_International_Episodes_Rescraped.csv')